# Gradients and initialization

The first part of this notebook describes the ***backpropagation algorithm***, which
computes these derivatives eﬀiciently. In the second part of the notebooks, we consider how to initialize the network parameters
before we commence training. We describe methods to choose the initial weights $\Omega_k$ and
biases $\beta_k$ so that training is stable.

**Optimization algorithms**:
The derivatives of the loss tell us how the loss changes when we make a small change
to the parameters. Optimization algorithms exploit this information to manipulate the
parameters so that the loss becomes smaller

**Observation 1**: Each weight (element of $\Omega_k$ ) multiplies the activation at a source hidden
unit and adds the result to a destination hidden unit in the next layer. It follows that the
effect of any small change to the weight is amplified or attenuated by the activation at
the source hidden unit. Hence, we run the network for each data example in the batch
and store the activations of all the hidden units. This is known as the forward pass. The stored activations will subsequently be used to compute the gradients.

In the context of training a deep learning model, "storing the activations" refers to keeping track of the intermediate outputs (or activations) of the hidden units (neurons) in a neural network during the forward pass.

The forward pass is the process of feeding an input (like a data example from your batch) through the network, layer by layer, to compute the final output (e.g., a prediction). As the data moves through each layer, every hidden unit calculates its activation based on the inputs it receives from the previous layer.

How It Works in Practice:
1. Forward Pass: For each data example in the batch, the network computes the activations of all hidden units, layer by layer, and stores them (e.g., in memory).
2. Backward Pass: Using these stored activations, the network calculates the gradients of the loss function with respect to each weight. This involves combining the stored activations with the error signals propagated backward through the network.
3. Weight Update: The gradients are then used to adjust the weights (e.g., via gradient descent) to reduce the error.

**Observation 2**:
A small change in a bias or weight causes a ripple effect of changes
through the subsequent network. The change modifies the value of its destination hidden.

As we move backward through the network, we see that most of the terms we need
were already calculated in the previous step, so we do not need to re-compute them.
Proceeding backward through the network in this way to compute the derivatives is
known as the **backward pass**.

The backpropagation algorithm is an eﬀicient method for computing all of these
derivatives at once. It consists of ($i$) a forward pass, in which we compute and store a
series of intermediate values and the network output, and ($ii$) a backward pass, in which we calculate the derivatives of each parameter, starting at the end of the network, and
reusing previous calculations as we move toward the start.

the whole process of backward pass and applying chain rule:
<br>
<br>
$$\frac{\partial l_i}{\partial \beta_k} = \frac{\partial f_k}{\partial \beta_k} \frac{\partial l_i}{\partial f_k}$$
<br>
$$\frac{\partial l_i}{\partial w_k} = \frac{\partial f_k}{\partial w_k} \frac{\partial l_i}{\partial f_k}$$

Assume we have a neural net with 3 layers as:


$$
\begin{align*}
\mathbf{f}_0 &= \beta_0 + \Omega_0 \mathbf{x}_i, \\
\mathbf{h}_1 &= \mathbf{a}[\mathbf{f}_0], \\
\mathbf{f}_1 &= \beta_1 + \Omega_1 \mathbf{h}_1, \\
\mathbf{h}_2 &= \mathbf{a}[\mathbf{f}_1], \\
\mathbf{f}_2 &= \beta_2 + \Omega_2 \mathbf{h}_2, \\
\mathbf{h}_3 &= \mathbf{a}[\mathbf{f}_2], \\
\mathbf{f}_3 &= \beta_3 + \Omega_3 \mathbf{h}_3, \\
\ell_i &= \mathbf{l}[\mathbf{f}_3, \mathbf{y}_i],
\end{align*}
$$

just for reference:
$$\begin{align*}
\frac{\partial \ell_i}{\partial f_2} &= \frac{\partial h_3}{\partial f_2} \left( \frac{\partial \ell_i}{\partial f_3} \right) \\
\frac{\partial \ell_i}{\partial h_2} &= \frac{\partial f_2}{\partial h_2} \left( \frac{\partial \ell_i}{\partial f_2} \frac{\partial f_2}{\partial h_3} \frac{\partial \ell_i}{\partial f_3} \right) \\
\frac{\partial \ell_i}{\partial f_1} &= \frac{\partial h_2}{\partial f_1} \left( \frac{\partial \ell_i}{\partial h_2} \frac{\partial f_2}{\partial h_2} \frac{\partial \ell_i}{\partial f_2} \frac{\partial f_2}{\partial h_3} \frac{\partial \ell_i}{\partial f_3} \right) \\
\frac{\partial \ell_i}{\partial h_1} &= \frac{\partial f_1}{\partial h_1} \left( \frac{\partial \ell_i}{\partial f_1} \frac{\partial h_2}{\partial f_1} \frac{\partial \ell_i}{\partial h_2} \frac{\partial f_2}{\partial h_2} \frac{\partial \ell_i}{\partial f_2} \frac{\partial f_2}{\partial h_3} \frac{\partial \ell_i}{\partial f_3} \right) \\
\frac{\partial \ell_i}{\partial f_0} &= \frac{\partial h_1}{\partial f_0} \left( \frac{\partial \ell_i}{\partial h_1} \frac{\partial f_1}{\partial h_1} \frac{\partial \ell_i}{\partial f_1} \frac{\partial h_2}{\partial f_1} \frac{\partial \ell_i}{\partial h_2} \frac{\partial f_2}{\partial h_2} \frac{\partial \ell_i}{\partial f_2} \frac{\partial f_2}{\partial h_3} \frac{\partial \ell_i}{\partial f_3} \right)
\end{align*}$$

in our previous declared neural-nets:
The derivative $\frac{\partial h_3}{\partial f_2}$ of the output $h_3$ of the activation function with respect to
its input $f_2$ will depend on the activation function. It will be a ***diagonal matrix***
since each activation only depends on the corresponding pre-activation. For ReLU
functions, the diagonal terms are zero everywhere $f_2$ is less than zero and one otherwise. Rather than multiply by this matrix, we extract the diagonal
terms as a vector $I[f_2 \gt 0]$ and pointwise multiply, which is more eﬀicient.
As
we progress back through the network, we alternately (i) multiply by the transpose of
the weight matrices $\Omega_{k}^T$ and (ii) threshold based on the inputs $f_k −1$ to the hidden layer.
These inputs were stored during the forward pass.

$$\
\frac{\partial \mathbf{f}_3}{\partial \mathbf{h}_3} = \frac{\partial}{\partial \mathbf{h}_3} \left( \beta_3 + \Omega_3 \mathbf{h}_3 \right) = \Omega_3^T.
$$

calculating the loss with respect to the Biases($\beta$) and weights($\Omega$):
<br>
<br>

$$
\begin{align*}
\frac{\partial \ell_i}{\partial \beta_k} &= \frac{\partial \mathbf{f}_k}{\partial \beta_k} \frac{\partial \ell_i}{\partial \mathbf{f}_k} \\
&= \frac{\partial}{\partial \beta_k} \left( \beta_k + \Omega_k \mathbf{h}_k \right) \frac{\partial \ell_i}{\partial \mathbf{f}_k} \\
&= \frac{\partial \ell_i}{\partial \mathbf{f}_k}, \\[2ex]
\frac{\partial \ell_i}{\partial \Omega_k} &= \frac{\partial \mathbf{f}_k}{\partial \Omega_k} \frac{\partial \ell_i}{\partial \mathbf{f}_k} \\
&= \frac{\partial}{\partial \Omega_k} \left( \beta_k + \Omega_k \mathbf{h}_k \right) \frac{\partial \ell_i}{\partial \mathbf{f}_k} \\
&= \frac{\partial \ell_i}{\partial \mathbf{f}_k} \mathbf{h}_k^T.
\end{align*}
$$

summerization of backpropagation algorithm:
Consider a deep neural
network $f[x_i,\phi]$ that takes input $x_i$, has $K$ hidden layers with ReLU activations, and
individual loss term $l_i = l[f[x_i,\phi],yi]$. The goal of backpropagation is to compute the
derivatives $\frac{\partial l_i}{\partial \beta_k}$ and $\frac{\partial l_i}{\partial \Omega_k}$ with respect to the biases $\beta_k$ and weights $\Omega_k$

Backward pass equations:
$$
\begin{align*}
\frac{\partial \ell_i}{\partial \beta_k} &= \frac{\partial \ell_i}{\partial \mathbf{f}_k} & k \in \{ K, K-1, \ldots, 1 \} \\
\frac{\partial \ell_i}{\partial \Omega_k} &= \frac{\partial \ell_i}{\partial \mathbf{f}_k} \mathbf{h}_k^T & k \in \{ K, K-1, \ldots, 1 \} \\
\frac{\partial \ell_i}{\partial \mathbf{f}_{k-1}} &= I[\mathbf{f}_{k-1} > 0] \odot \left( \Omega_k^T \frac{\partial \ell_i}{\partial \mathbf{f}_k} \right), & k \in \{ K, K-1, \ldots, 1 \} \\[2ex]
\frac{\partial \ell_i}{\partial \beta_0} &= \frac{\partial \ell_i}{\partial \mathbf{f}_0} \\
\frac{\partial \ell_i}{\partial \Omega_0} &= \frac{\partial \ell_i}{\partial \mathbf{f}_0} \mathbf{x}_i^T.
\end{align*}
$$

where $\odot$ denotes pointwise multiplication, and $I[f_k −1 > 0]$ is a vector containing ones
where $f_k −1$ is greater than zero and zeros elsewhere.
<br>
We calculate these derivatives for every training example in the batch and sum them
together to retrieve the gradient for the SGD update

Note that the backpropagation algorithm is extremely eﬀicient; the most demanding
computational step in both the forward and backward pass is matrix multiplication (by $\Omega$
and $\Omega^T$ , respectively) which only requires additions and multiplications. However, it is
not memory eﬀicient; the intermediate values in the forward pass must all be stored, and
this can limit the size of the model we can train.

### Parameter initialization:
#### ( vanishing gradient problem, exploding gradient problem )


$$\begin{align*}
\mathbb{E}[f'_i] &= \mathbb{E} \left[ \beta_i + \sum_{j=1}^{D_h} \Omega_{i,j} h_j \right] \\
&= \mathbb{E}[\beta_i] + \sum_{j=1}^{D_h} \mathbb{E}[\Omega_{i,j} h_j] \\
&= \mathbb{E}[\beta_i] + \sum_{j=1}^{D_h} \mathbb{E}[\Omega_{i,j}] \mathbb{E}[h_j] \\
&= 0 + \sum_{j=1}^{D_h} 0 \cdot \mathbb{E}[h_j] = 0, \\
\sigma^2_{f'_i} &= \mathbb{E}[f'^2_i] - \mathbb{E}[f'_i]^2 \\
&= \mathbb{E} \left[ \left( \beta_i + \sum_{j=1}^{D_h} \Omega_{i,j} h_j \right)^2 \right] - 0 \\
&= \mathbb{E} \left[ \left( \sum_{j=1}^{D_h} \Omega_{i,j} h_j \right)^2 \right] \\
&= \sum_{j=1}^{D_h} \mathbb{E}[\Omega^2_{i,j}] \mathbb{E}[h^2_j] \\
&= \sum_{j=1}^{D_h} \sigma^2_\Omega \mathbb{E}[h^2_j] = \sigma^2_\Omega \sum_{j=1}^{D_h} \mathbb{E}[h^2_j],
\end{align*}$$

and finally with the ReLU activation function (which clips the values less that zero), we get:
$$\sigma^2_{f'_i} = \sigma^2_\Omega \sum_{j=1}^{D_h} \frac{\sigma^2_{f_j}}{2} = \frac{1}{2} D_h \sigma^2_\Omega \sigma^2_f.$$

This, in turn, implies that if we want the variance $\sigma_{f'}^2$ of the subsequent pre-activations $f'$
to be the same as the variance $\sigma_f^2$ of the original pre-activations $f$ during the forward
pass, we should set:
$$\sigma^2_\Omega = \frac{2}{D_h}$$

where $D_h$ is the dimension of the original layer to which the weights were applied. This
is known as ***He initialization***

but what if the hidden units in consequitive layers differs:
$$\sigma_\Omega^2 = \frac{4}{D_h + D_{h'}}$$

In [ ]:
import torch, torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import StepLR

# define input size, hidden layer size, output size
D_i, D_k, D_o = 10, 40, 5

# create model with two hidden layers
model = nn.Sequential(
  nn.Linear(D_i, D_k),
  nn.ReLU(),
  nn.Linear(D_k, D_k),
  nn.ReLU(),
  nn.Linear(D_k, D_o))

# He initialization of weights
def weights_init(layer_in):
  if isinstance(layer_in, nn.Linear):
    nn.init.kaiming_normal_(layer_in.weight)
    layer_in.bias.data.fill_(0.0)

model.apply(weights_init)

# choose least squares loss function
criterion = nn.MSELoss()

# construct SGD optimizer and initialize learning rate and momentum
optimizer = torch.optim.SGD(model.parameters(), lr = 0.1, momentum=0.9)

# object that decreases learning rate by half every 10 epochs
scheduler = StepLR(optimizer, step_size=10, gamma=0.5)

# create 100 random data points and store in data loader class
x = torch.randn(100, D_i)
y = torch.randn(100, D_o)
data_loader = DataLoader(TensorDataset(x,y), batch_size=10, shuffle=True)

# loop over the dataset 100 times
for epoch in range(100):
  epoch_loss = 0.0
  # loop over batches
  for i, data in enumerate(data_loader):
    # retrieve inputs and labels for this batch
    x_batch, y_batch = data
    # zero the parameter gradients
    optimizer.zero_grad()
    # forward pass
    pred = model(x_batch)
    loss = criterion(pred, y_batch)
    # backward pass
    loss.backward()
    # SGD update
    optimizer.step()
    # update statistics
    epoch_loss += loss.item()
  # print error
  print(f'Epoch {epoch:5d}, loss {epoch_loss:.3f}')
  # tell scheduler to consider updating learning rate
  scheduler.step()

Epoch     0, loss 14.346
Epoch     1, loss 8.229
Epoch     2, loss 7.697
Epoch     3, loss 6.941
Epoch     4, loss 6.468
Epoch     5, loss 6.067
Epoch     6, loss 5.638
Epoch     7, loss 5.403
Epoch     8, loss 5.219
Epoch     9, loss 4.992
Epoch    10, loss 4.684
Epoch    11, loss 4.106
Epoch    12, loss 3.780
Epoch    13, loss 3.473
Epoch    14, loss 3.313
Epoch    15, loss 3.091
Epoch    16, loss 2.969
Epoch    17, loss 2.906
Epoch    18, loss 2.790
Epoch    19, loss 2.765
Epoch    20, loss 2.680
Epoch    21, loss 2.511
Epoch    22, loss 2.476
Epoch    23, loss 2.416
Epoch    24, loss 2.373
Epoch    25, loss 2.344
Epoch    26, loss 2.299
Epoch    27, loss 2.291
Epoch    28, loss 2.253
Epoch    29, loss 2.222
Epoch    30, loss 2.184
Epoch    31, loss 2.159
Epoch    32, loss 2.145
Epoch    33, loss 2.129
Epoch    34, loss 2.112
Epoch    35, loss 2.108
Epoch    36, loss 2.098
Epoch    37, loss 2.083
Epoch    38, loss 2.076
Epoch    39, loss 2.067
Epoch    40, loss 2.045
Epoch    41, lo

further research:
- Activation normalization (ActNorm)?
- Layer-sequential unit variance initialization?
- Batch-Norm
- *ConvolutionOrthogonal* initializer
- *Fixup* for residual networks
- synchronous training in data parallelism

# Problem 7.6

Show that for $\mathbf{z} = \beta + \Omega \mathbf{h}$:

$$
\frac{\partial \mathbf{z}}{\partial \mathbf{h}} = \Omega^{\top},
$$

where $\frac{\partial \mathbf{z}}{\partial \mathbf{h}}$ is a matrix containing the term $\frac{\partial z_i}{\partial h_j}$ in its $i$-th column and $j$-th row. To do this, first find an expression for the constituent elements $\frac{\partial z_i}{\partial h_j}$, and then consider the form that the matrix $\frac{\partial \mathbf{z}}{\partial \mathbf{h}}$must take.

## Answer

We have:

$$
z_i = \beta_i + \sum_j w_{ij} h_j
$$

and so when we take the derivative, we get:

$$
\frac{\partial z_i}{\partial h_j} = w_{ij}
$$

This will be at the $i$-th column and $j$-th row (i.e., at position $(j,i)$ of the matrix $\frac{\partial \mathbf{z}}{\partial \mathbf{h}}$, which is hence $\Omega^{\top}$.

### totally in backprop:

imagine:


\begin{align}
f_{0} &=& \beta_{0} + \omega_{0} x_i\nonumber\\
h_{1} &=& \sin[f_{0}]\nonumber\\
f_{1} &=& \beta_{1} + \omega_{1}h_{1}\nonumber\\
h_{2} &=& \exp[f_{1}]\nonumber\\
f_{2} &=& \beta_{2} + \omega_{2} h_{2}\nonumber\\
h_{3} &=& \cos[f_{2}]\nonumber\\
f_{3} &=& \beta_{3} + \omega_{3}h_{3}\nonumber\\
l_i &=& (f_3-y_i)^2
\end{align}

<br>

we want:

\begin{align}
\quad \frac{\partial \ell_i}{\partial f_3}, \quad \frac{\partial \ell_i}{\partial h_3}, \quad \frac{\partial \ell_i}{\partial f_2}, \quad
\frac{\partial \ell_i}{\partial h_2}, \quad \frac{\partial \ell_i}{\partial f_1}, \quad \frac{\partial \ell_i}{\partial h_1},  \quad\text{and} \quad \frac{\partial \ell_i}{\partial f_0}.
\end{align}

<br>

so we get:

\begin{align}
\frac{\partial \ell_i}{\partial f_{2}} &=& \frac{\partial h_{3}}{\partial f_{2}}\left(
\frac{\partial f_{3}}{\partial h_{3}}\frac{\partial \ell_i}{\partial f_{3}} \right)
\nonumber \\
\frac{\partial \ell_i}{\partial h_{2}} &=& \frac{\partial f_{2}}{\partial h_{2}}\left(\frac{\partial h_{3}}{\partial f_{2}}\frac{\partial f_{3}}{\partial h_{3}}\frac{\partial \ell_i}{\partial f_{3}}\right)\nonumber \\
\frac{\partial \ell_i}{\partial f_{1}} &=& \frac{\partial h_{2}}{\partial f_{1}}\left( \frac{\partial f_{2}}{\partial h_{2}}\frac{\partial h_{3}}{\partial f_{2}}\frac{\partial f_{3}}{\partial h_{3}}\frac{\partial \ell_i}{\partial f_{3}} \right)\nonumber \\
\frac{\partial \ell_i}{\partial h_{1}} &=& \frac{\partial f_{1}}{\partial h_{1}}\left(\frac{\partial h_{2}}{\partial f_{1}} \frac{\partial f_{2}}{\partial h_{2}}\frac{\partial h_{3}}{\partial f_{2}}\frac{\partial f_{3}}{\partial h_{3}}\frac{\partial \ell_i}{\partial f_{3}} \right)\nonumber \\
\frac{\partial \ell_i}{\partial f_{0}} &=& \frac{\partial h_{1}}{\partial f_{0}}\left(\frac{\partial f_{1}}{\partial h_{1}}\frac{\partial h_{2}}{\partial f_{1}} \frac{\partial f_{2}}{\partial h_{2}}\frac{\partial h_{3}}{\partial f_{2}}\frac{\partial f_{3}}{\partial h_{3}}\frac{\partial \ell_i}{\partial f_{3}} \right).
\end{align}
